In [0]:
service_credential = dbutils.secrets.get(scope="analyticssecretscope",key="analyticsecret")

spark.conf.set("fs.azure.account.auth.type.candlestickstr.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.candlestickstr.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.candlestickstr.dfs.core.windows.net", "682f15f2-5550-44d1-a25b-51fec5774443")
spark.conf.set("fs.azure.account.oauth2.client.secret.candlestickstr.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.candlestickstr.dfs.core.windows.net", "https://login.microsoftonline.com/785087ba-1e72-4e7d-b1d1-4a9639137a66/oauth2/token")

print("CONNECTION OK")

In [0]:
# 마운트 포인트 및 기대하는 소스 정의
MOUNT_POINT = "/mnt/my-mount"
EXPECTED_SOURCE = "abfss://candlestick2024@candlestickstr.dfs.core.windows.net/"

# OAuth 인증 구성
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "682f15f2-5550-44d1-a25b-51fec5774443",
    "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="analyticssecretscope", key="analyticsecret"),
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/785087ba-1e72-4e7d-b1d1-4a9639137a66/oauth2/token"
}

# 마운트 상태 확인 및 잘못 연결된 경우 재마운트
is_mounted = False
for mount in dbutils.fs.mounts():
    if mount.mountPoint == MOUNT_POINT:
        is_mounted = True
        if mount.source != EXPECTED_SOURCE:
            print(f"⚠️ 잘못된 소스로 마운트되어 있음: {mount.source}")
            print("🔄 기존 마운트 해제 중...")
            dbutils.fs.unmount(MOUNT_POINT)
            is_mounted = False  # 다시 mount 필요
        else:
            print(f"✅ 이미 올바른 소스로 마운트되어 있음: {mount.source}")
        break

# 마운트가 안 되어 있거나 해제된 경우 새로 마운트
if not is_mounted:
    print("🚀 마운트 시작...")
    dbutils.fs.mount(
        source=EXPECTED_SOURCE,
        mount_point=MOUNT_POINT,
        extra_configs=configs
    )
    print("✅ 마운트 완료:", MOUNT_POINT)


In [0]:

# 실행 날짜 위젯
dbutils.widgets.text("execution_date", "2024-01-04")
execution_date = dbutils.widgets.get("execution_date")


# bronze 계층 증분 데이터 로드
from pyspark.sql.functions import col, to_date

execution_date = "2024-01-04"

for market in ["KRW-BTC", "KRW-ETH", "KRW-XRP"]:
    table_name = f"bronze.bronze_{market.lower().replace('-', '_')}"
    json_path = f"/mnt/my-mount/{market}/{market}-{execution_date}.json"

    print(table_name)
    print(json_path)

    try:
        # 원본 JSON 읽기
        bronze_df = spark.read.option("multiline", "true").json(json_path)
        bronze_df.show(truncate=False)
        print("*" * 100)

        # 날짜 존재 여부 확인
        is_exist = (
            spark.table(table_name)
            .filter(to_date(col("candle_date_time_kst")) == execution_date)
            .limit(1)
            .count()
            > 0
        )

        if is_exist:
            print(f"✅ {market} - 날짜 {execution_date} 삭제 후 append")

            # 먼저 해당 날짜 삭제
            spark.sql(
                f"""
                DELETE FROM {table_name}
                WHERE to_date(candle_date_time_kst) = DATE('{execution_date}')
                """
            )

        else:
            print(f"🆕 {market} - 날짜 {execution_date} append")

        # 공통: 데이터 추가
        bronze_df.write.format("delta").mode("append").saveAsTable(table_name)

    except Exception as e:
        print(f"❌ {market} 처리 실패: {e}")
